In [1]:
# Bibliotecas 
import pandas as pd
import torch as torch
import numpy as np
import math
import requests
import random

import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits import mplot3d
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier  
from sklearn import preprocessing
from sklearn import utils
from matplotlib.colors import ListedColormap 
import datetime as dt
from numpy import arange
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings("ignore")

Hipótesis: predicir precio de cierre de 5 compañías del indice de acciones NASQAD: Google, Twitter, Tesla, Amazon y Apple.

## Preprocesado

In [2]:
df = pd.read_csv('NASDAQ_100_Data_From_2010.csv', sep = '\t')

In [3]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Name
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.562591,493729600,AAPL
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.573935,601904800,AAPL
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.469369,552160000,AAPL
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.457407,477131200,AAPL
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.500339,447610800,AAPL


In [4]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Name
271675,2021-09-03,295.325012,301.804993,292.029999,298.290009,298.290009,6127900,ZM
271676,2021-09-07,298.295013,300.980011,294.799988,299.959991,299.959991,4251900,ZM
271677,2021-09-08,299.549988,299.959991,290.529999,293.600006,293.600006,3934400,ZM
271678,2021-09-09,292.160004,297.570007,291.130005,295.859985,295.859985,3350100,ZM
271679,2021-09-10,296.910004,306.263000,296.809998,301.500000,301.500000,6089600,ZM


In [5]:
df_describe = pd.DataFrame(df)
df_describe.describe()

,Open,High,Low,Close,Adj Close,Volume
count,271680.000000,271680.000000,271680.000000,271680.000000,271680.000000,2.716800e+05
mean,130.147060,131.678573,128.564517,130.173960,126.929715,1.052670e+07
std,259.463324,262.249242,256.522832,259.455010,260.156874,3.924802e+07
min,0.610000,0.660000,0.610000,0.650000,0.612270,0.000000e+00
25%,32.549999,32.950001,32.150002,32.570000,28.001979,1.332175e+06
50%,59.810001,60.504999,59.119999,59.849998,55.599998,2.759400e+06
75%,117.139999,118.470001,115.820000,117.190002,114.705492,6.889500e+06
max,3744.000000,3773.080078,3696.790039,3731.409912,3731.409912,1.880998e+09


In [6]:
list(df.columns.values)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Name']

In [7]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
Name         0
dtype: int64

In [8]:
df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
Name          object
dtype: object

In [9]:
df.drop(["Volume"], axis=1, inplace=True)

In [10]:
list(df.columns.values)

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Name']

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271680 entries, 0 to 271679
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Date       271680 non-null  object 
 1   Open       271680 non-null  float64
 2   High       271680 non-null  float64
 3   Low        271680 non-null  float64
 4   Close      271680 non-null  float64
 5   Adj Close  271680 non-null  float64
 6   Name       271680 non-null  object 
dtypes: float64(5), object(2)
memory usage: 14.5+ MB


In [12]:
df.shape

(271680, 7)

In [13]:
df.ndim

2

In [14]:
df.set_index('Date', inplace=True)
df.dropna(inplace=True)

#### Filtración de datos para reducir la muestra:

En el siguiente bloque de código vamos a filtrar el dataset para mostrar solos los datos de las cinco empresas seleccionadas:

- Cisco Systems, Inc.: 'CSCO'
- Google Inc.: 'GOOG'
- Tesla, Inc.: 'TSLA'
- Microsoft Corporation: 'MSFT'
- Amazon.com, Inc.: 'AMZN'

Se busca reducir el número de columnas, que actualmente es de 271680 para poder hacer el análisis de resultados más entendible y manejable. 

In [ ]:
selection = ['CSCO','GOOG', 'TSLA','AMZN','MSFT']
short_df = df[df['Close'].isin(selection)]
print(short_df)

In [16]:
df = short_df
df.shape

## EDA

## Diagrama de Pares: Observaciones

Al relacionar los valores de las columnas de precio de apartura "Open", precio de cierre "Close", precio máximo alcanzado en el día "High", precio mínimo "Low", precio de cierre ajustado "Adj Close" vemos linealidad, lo que significa que los datos tienen una variación muy reducida y siguen la tendencia del movimiento sin valores extremos. Esto es diferente para las relaciones con el volumen, dado que el volumen es el número de acciones negociadas en ese día, por lo que vemos valores extremos que se alejan del grueso de datos en las partes superiores, respondiendo a la volatilidad particular por eventos en el mercado como noticias financieras o rumores acerca del valor futuro de ciertas acciones. 

In [ ]:
sns.pairplot(df);

## Mapa de Calor: Observaciones

Donde el número representa la línealidad y congruencia en el comportamiento de las acciones, sin valores extremos siguen la misma tendencia. Significa que hay correlación en la variables, lo que hace los datos útiles para ser usados en un modelo de apredizaje automático usando el algoritmo Random Forest.

In [ ]:
#Mapa de calor 
def correlation_heatmap(df1):
    _,ax=plt.subplots(figsize=(15,10))
    colormap=sns.diverging_palette(220,10,as_cmap=True)
    sns.heatmap(df.corr(),annot=True,cmap=colormap)
    
correlation_heatmap(df)

## Observaciones: Gráfico de Dispersión

Resulta interesante observar que hay tres empresas: Tesla, Microsoft y Cisco, donde el volumen de intercambios diarios es mayor, por lo tanto los precios tienden a ser más volátiles. 

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(x='Volume',y='Name',data=df)
plt.xlabel('Volume')
plt.ylabel('Compañía')

## Desviación Estandar del precio de cierre

In [ ]:
print("========================================================")
print("Desviación estandar: variables de las acciones de Tesla")
df_TSLA = df[df['Name'] == 'TSLA']
sd_TSLA = np.std(df_TSLA)
print((sd_TSLA))
print("========================================================")
print("Desviación estandar: variables de las acciones de Cisco")
df_CSCO = df[df['Name'] == 'CSCO']
sd_CSCO = np.std(df_CSCO)
print(round(sd_CSCO))
print("========================================================")
print("Desviación estandar: variables de las acciones de Google")
df_GOOG = df[df['Name'] == 'GOOG']
sd_GOOG = np.std(df_GOOG)
print((sd_GOOG))
print("========================================================")
print("Desviación estandar: variables de las acciones de Amazon")
df_AMZN = df[df['Name'] == 'AMZN']
sd_AMZN = np.std(df_AMZN)
print((sd_AMZN))
print("========================================================")
print("Desviación estandar: variables de las acciones de Microsoft")
df_MSFT = df[df['Name'] == 'MSFT']
sd_MSFT = np.std(df_MSFT)
print((sd_MSFT))


In [ ]:
df.set_index("Date", inplace=True)
df['Adj Close'].plot()
plt.ylabel("Adjusted Close Prices")
plt.show()

## Modelo: Random Forest

### Paso 1:
Seleccionar las columnas que son númericas y que sirven para entrenar el modelo, define los datos de prueba y entrenamiento

In [ ]:
column_selection = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
x = df.iloc[:,[2,3]].values
y = df.iloc[:,4].values

In [ ]:
#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y = lab.fit_transform(y)

#view transformed values
print(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.25, random_state=0)  

In [ ]:
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)   

### Paso 2: 

Construir los árboles de decisión

In [ ]:
classifier = RandomForestClassifier(n_estimators= 10, criterion="entropy")  
classifier.fit(x_train, y_train)  

In [ ]:
y_pred= classifier.predict(x_test)  

print(y_pred)

### Confusion Matrix

In [ ]:
cm= confusion_matrix(y_test, y_pred)

cm

In [ ]:
x_set, y_set = x_train, y_train  
x1, x2 = np.meshgrid(np.arange(start = x_set[:, 0].min() - 1, stop = x_set[:, 0].max() + 1, step  =0.01),  
np.arange(start = x_set[:, 1].min() - 1, stop = x_set[:, 1].max() + 1, step = 0.01))  
plt.contourf(x1, x2, classifier.predict(np.array([x1.ravel(), x2.ravel()]).T).reshape(x1.shape),  
alpha = 0.75, cmap = ListedColormap(('purple','green' )))  
plt.xlim(x1.min(), x1.max())  
plt.ylim(x2.min(), x2.max())  
for i, j in enumerate(np.unique(y_set)):  
    plt.scatter(x_set[y_set == j, 0], x_set[y_set == j, 1],  
        c = ListedColormap(('purple', 'green'))(i), label = j)  
plt.title('Random Forest Algorithm (Training set)')  
plt.xlabel('Age')  
plt.ylabel('Estimated Salary')  
plt.legend()  
plt.show()  

## Métricas

- MAPE
- RMSE
- MBE